In [2]:
K.<a> = NumberField(x - 1)
from sage.rings.polynomial.cyclotomic import cyclotomic_coeffs
S = ZZ['x']
K.<w> = NumberField(x^2 - x + 1)
R.<x> = PolynomialRing(ZZ)
from collections import defaultdict


def is_relatively_prime(m,n):
    if gcd(m,n) ==1:
        return 1
    else:
        return 0 
    
def raw_fekete(n):
    "Fekete polynomial with Delta odd "
    D=n
    v=[is_relatively_prime(a+1,n) for a in range(0,D-1)]
    F_D=R(v)
    return F_D*x  

def fekete_by_last_factor(n):
    F=raw_fekete(n)
    f = F.factor()[-1][0]
    return f


def divisor_set_general(q,p):
    divisor = set()
    for d in range(2, q):
        if q%d ==1:
            divisor.add(d)
    for d in range(2,p):
        if p%d==1 and d !=q:
            divisor.add(d)
    for d in range(2,p+q+1):
        if (q*p+1)%d==0 and (p+q)%d==0:
            divisor.add(d)
    return divisor 

def cyclotomic_factor(q,p):
    """
    return the cyclotmic factor of F_{qp}
    """
    cyc_factor =x
    divisor_set = divisor_set_general(q,p)
    for d in divisor_set:
        cyc_factor *=S(cyclotomic_coeffs(d))
    return cyc_factor

def fekete(q,p):
    n=p*q
    cyc_factor = cyclotomic_factor(q,p)
    F= raw_fekete(n)
    f,r =F.quo_rem(cyc_factor) 
    return f 

def middle_coef(f):
    coefs = f.coefficients(sparse=True)
    deg = len(coefs)
    half_deg = deg //2 
    return coefs[half_deg]

def max_coef(n):
    f=fekete(n)
    coefs = f.coefficients(sparse=True)
    abs_coefs = [abs(a) for a in coefs]
    return max(abs_coefs)

        
def reduced_fekete(f):
    u=f.trace_polynomial()
    g_D=u[0]
    return g_D

def fekete_reduction(f, q):
    f=f.change_ring(GF(q))
    return f.factor()         
                
def irreducible(f,n):
    for q in range(n):
        if is_prime(q): 
            factor=fekete_reduction(f,q)
            if len(factor)==1 and factor[0][1]==1:
                    return q
    return  -1         
               

def test_irreducible(f,q):
    factors = fekete_reduction(f,q)
    d=defaultdict(int)
    for factor in factors:
        degree = factor[0].degree()
        d[degree] += factor[1]
    for degree, count in d.items():
        if count % 2 ==1:
            return True
    return False    

def find_smallest_q1(f,n):
    for q in range(2,n):
        if is_prime(q) and test_irreducible(f,q):
            return q
    return  -1       

def finding_pair_q1_q2(f,n):
    g=f.trace_polynomial()[0]
    q2 = irreducible(g,n)
    q1 = find_smallest_q1(f,n)
    return [q1, q2]

def irreducible_test(f,n):
    pair = finding_pair_q1_q2(f,n)
    if pair[0] !=-1 and pair[1] !=-1:
        return True
    return False
    
    

# Finding the pair $(q_1, q_2)$ such that 

1. $g_{pq}$ is irreducible modulo $q_2$. 

2. In the factorization of $f_{pq}$ modulo $q_1$, there exists a degree that appears an odd number of times. 



We explain the strategy with a \(random\) example, say $q=11$ and $p=37$. First we compute $f_{pq}$. 


In [3]:
q=5
p=7
f = fekete(q,p)
f

x^22 - x^18 + x^17 - x^16 + x^15 - x^13 + x^12 + x^10 - x^9 + x^7 - x^6 + x^5 - x^4 + 1

We can also compute $g_{pq}$ 



In [4]:
g = reduced_fekete(f)
g

x^11 - 11*x^9 + 43*x^7 + x^6 - 71*x^5 - 5*x^4 + 46*x^3 + 4*x^2 - 8*x + 2

The following function returns \(q\_1, q\_2\). Here we restrict $q_1, q_2 \leq n$ with $n=10^5$ 


In [9]:
n=10**5
finding_pair_q1_q2(f,n)

[2, 29]

Let us test that this indeed works. First let us consider $f_{pq}$ modulo $q_1=2$. 



In [10]:
fekete_reduction(f,2)

(x + 1)^8 * (x^2 + x + 1)^2 * (x^3 + x + 1) * (x^3 + x^2 + 1) * (x^4 + x^3 + x^2 + x + 1)

We see for example that degree 4 appears only one time.

Let us consider $g_{pq}$ modulo $q_2=29$. 



In [8]:
fekete_reduction(g,29)

x^11 + 18*x^9 + 14*x^7 + x^6 + 16*x^5 + 24*x^4 + 17*x^3 + 4*x^2 + 21*x + 2

We see that it is indeed irreducible. So, we conclude that $f$ is irreducible as well.

# With the following blocks of code, we check that $f_{pq}$ is irreducible for $3 \leq q <p \leq 50$



In [3]:
P=Primes()
q_list = []
q=11
while q<100:
    q=P.next(q)
    q_list.append(q)
print(q_list)    

[13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101]


In [7]:
%%time
irreducible_finding =[]
n=10**5
for q in q_list:
    p=P.next(q)
    while p * q < 1000: 
        f=fekete(q,p)
        irreducible_finding.append([q,p,finding_pair_q1_q2(f,n)])
        p=P.next(p)              
print(irreducible_finding)                           
        

[[13, 17, [2, 449]], [13, 19, [2, 761]], [13, 23, [2, 19]], [13, 29, [2, 313]], [13, 31, [2, 2969]], [13, 37, [2, 457]], [13, 41, [2, 1039]], [13, 43, [2, 5]], [13, 47, [2, 607]], [13, 53, [2, 991]], [13, 59, [2, 2311]], [13, 61, [2, 631]], [13, 67, [2, 409]], [13, 71, [2, 1597]], [13, 73, [2, 2687]], [17, 19, [2, 127]], [17, 23, [5, 31]], [17, 29, [2, 2647]], [17, 31, [3, 263]], [17, 37, [2, 569]], [17, 41, [3, 5399]], [17, 43, [2, 311]], [17, 47, [3, 463]], [17, 53, [2, 7789]], [19, 23, [2, 37]], [19, 29, [2, 1931]], [19, 31, [2, 1303]], [19, 37, [2, 353]], [19, 41, [2, 1187]], [19, 43, [2, 6857]], [19, 47, [2, 2777]], [23, 29, [2, 43]], [23, 31, [3, 3221]], [23, 37, [2, 1949]], [23, 41, [5, 853]], [23, 43, [2, 107]], [29, 31, [2, 1213]]]
CPU times: user 3min 59s, sys: 760 ms, total: 4min
Wall time: 12min 10s


In [13]:
%%time
irreducible_finding =[]
n=10**5
lower_range = 1000
upper_range = 2000
for q in q_list:
    p=max(P.next(q), P.next(lower_range//q))    
    while p * q < upper_range: 
        f=fekete(q,p)
        irreducible_finding.append([q,p,finding_pair_q1_q2(f,n)])
        p=P.next(p)              
print(irreducible_finding)                           
        

[[13, 79, [3, 6703]], [13, 83, [2, 359]], [13, 89, [2, 4861]], [13, 97, [2, 5839]], [13, 101, [2, 4987]], [13, 103, [2, 12347]], [13, 107, [2, 4271]], [13, 109, [2, 419]], [13, 113, [2, 9403]], [13, 127, [2, 373]], [13, 131, [2, 1319]], [13, 137, [2, 4447]], [13, 139, [2, 6469]], [13, 149, [2, 919]], [13, 151, [2, 6883]], [17, 59, [2, 2699]], [17, 61, [2, 317]], [17, 67, [2, 3347]], [17, 71, [3, 5023]], [17, 73, [3, 503]], [17, 79, [3, 257]], [17, 83, [2, 1259]], [17, 89, [3, 1531]], [17, 97, [3, 15361]], [17, 101, [2, 2467]], [17, 103, [3, 8329]], [17, 107, [2, 577]], [17, 109, [2, 1109]], [17, 113, [3, 33161]], [19, 53, [2, 1103]], [19, 59, [2, 5021]], [19, 61, [2, 227]], [19, 67, [2, 823]], [19, 71, [2, 5107]], [19, 73, [2, 1459]], [19, 79, [2, 241]], [19, 83, [2, 5333]], [19, 89, [2, 3391]], [19, 97, [2, 3803]], [19, 101, [2, 2797]], [19, 103, [2, 16417]], [23, 47, [3, 563]], [23, 53, [2, 1429]], [23, 59, [2, 2749]], [23, 61, [2, 7877]], [23, 67, [2, 6323]], [23, 71, [3, 563]], [23

In [15]:
%%time
irreducible_finding =[]
n=10**5
lower_range = 2000
upper_range = 3000
for q in q_list:
    p=max(P.next(q), P.next(lower_range//q))    
    while p * q < upper_range: 
        f=fekete(q,p)
        irreducible_finding.append([q,p,finding_pair_q1_q2(f,n)])
        p=P.next(p)              
print(irreducible_finding)                           
        

[[13, 157, [2, 3449]], [13, 163, [2, 877]], [13, 167, [2, 16127]], [13, 173, [2, 277]], [13, 179, [2, 2801]], [13, 181, [2, 109]], [13, 191, [2, 16249]], [13, 193, [2, 3581]], [13, 197, [2, 1931]], [13, 199, [2, 7127]], [13, 211, [2, 23]], [13, 223, [2, 30137]], [13, 227, [2, 9511]], [13, 229, [2, 2789]], [17, 127, [3, 7019]], [17, 131, [2, 6247]], [17, 137, [3, 13121]], [17, 139, [2, 4337]], [17, 149, [2, 127]], [17, 151, [3, 4447]], [17, 157, [2, 6113]], [17, 163, [2, 4657]], [17, 167, [3, 1811]], [17, 173, [2, 1619]], [19, 107, [2, 439]], [19, 109, [2, 6221]], [19, 113, [2, 9091]], [19, 127, [2, 5737]], [19, 131, [2, 28081]], [19, 137, [2, 1709]], [19, 139, [2, 6997]], [19, 149, [2, 2887]], [19, 151, [2, 3919]], [19, 157, [2, 19163]], [23, 89, [3, 7]], [23, 97, [3, 7331]], [23, 101, [2, 21163]], [23, 103, [3, 359]], [23, 107, [2, 211]], [23, 109, [2, 19273]], [23, 113, [3, 2423]], [23, 127, [3, 52817]], [29, 71, [2, 4021]], [29, 73, [2, 18457]], [29, 79, [2, 3631]], [29, 83, [2, 119

In [16]:
%%time
irreducible_finding =[]
n=10**5
lower_range = 3000
upper_range = 4000
for q in q_list:
    p=max(P.next(q), P.next(lower_range//q))    
    while p * q < upper_range: 
        f=fekete(q,p)
        irreducible_finding.append([q,p,finding_pair_q1_q2(f,n)])
        p=P.next(p)              
print(irreducible_finding)                           
        

[[13, 233, [2, 9521]], [13, 239, [2, 4003]], [13, 241, [2, 7607]], [13, 251, [2, 4357]], [13, 257, [2, 9907]], [13, 263, [2, 18233]], [13, 269, [2, 10973]], [13, 271, [2, 35797]], [13, 277, [2, 3323]], [13, 281, [2, 14321]], [13, 283, [2, 2069]], [13, 293, [2, 16067]], [13, 307, [2, 1259]], [17, 179, [2, 36767]], [17, 181, [2, 2437]], [17, 191, [3, 3919]], [17, 193, [3, 151]], [17, 197, [2, 9649]], [17, 199, [3, 21001]], [17, 211, [2, 7699]], [17, 223, [3, 3137]], [17, 227, [2, 24121]], [17, 229, [2, 12377]], [17, 233, [3, 20327]], [19, 163, [2, 8017]], [19, 167, [2, 1889]], [19, 173, [2, 19427]], [19, 179, [2, 18451]], [19, 181, [2, 2699]], [19, 191, [3, 3187]], [19, 193, [2, 33809]], [19, 197, [2, 1571]], [19, 199, [2, 9013]], [23, 131, [2, 20341]], [23, 137, [3, 31]], [23, 139, [2, 7309]], [23, 149, [2, 19583]], [23, 151, [3, 313]], [23, 157, [2, 9133]], [23, 163, [2, 18077]], [23, 167, [5, 33893]], [23, 173, [2, 13381]], [29, 107, [2, 33161]], [29, 109, [2, 7477]], [29, 113, [2, 40

In [17]:
%%time
irreducible_finding =[]
n=10**5
lower_range = 4000
upper_range = 4500
for q in q_list:
    p=max(P.next(q), P.next(lower_range//q))    
    while p * q < upper_range: 
        f=fekete(q,p)
        irreducible_finding.append([q,p,finding_pair_q1_q2(f,n)])
        p=P.next(p)              
print(irreducible_finding)                           
        

[[13, 311, [2, 14951]], [13, 313, [3, 7489]], [13, 317, [2, 44017]], [13, 331, [2, 53923]], [13, 337, [2, 42461]], [17, 239, [3, 1493]], [17, 241, [3, 26801]], [17, 251, [2, 66413]], [17, 257, [3, 15131]], [17, 263, [3, 2129]], [19, 211, [2, 263]], [19, 223, [2, 4001]], [19, 227, [2, 30347]], [19, 229, [2, 10753]], [19, 233, [2, 1777]], [23, 179, [2, 15641]], [23, 181, [2, 9769]], [23, 191, [3, 21617]], [23, 193, [3, 1493]], [29, 139, [2, 33713]], [29, 149, [2, 7963]], [29, 151, [2, 10193]], [31, 131, [2, 11633]], [31, 137, [3, 21683]], [31, 139, [2, 36997]], [37, 109, [3, 20809]], [37, 113, [2, 8081]], [41, 101, [2, 4801]], [41, 103, [3, 53]], [41, 107, [2, 11579]], [41, 109, [2, 2069]], [43, 97, [2, 16381]], [43, 101, [2, 26111]], [43, 103, [2, 33181]], [47, 89, [3, 47293]], [53, 79, [2, 23087]], [53, 83, [2, 6833]], [59, 71, [2, 25561]], [59, 73, [2, 12239]], [61, 67, [2, 5897]], [61, 71, [2, 503]], [61, 73, [2, 8387]]]
CPU times: user 1d 14h 1min 58s, sys: 6min 18s, total: 1d 14h 8

In [18]:
%%time
irreducible_finding =[]
n=10**5
lower_range = 4500
upper_range = 5000
for q in q_list:
    p=max(P.next(q), P.next(lower_range//q))    
    while p * q < upper_range: 
        f=fekete(q,p)
        irreducible_finding.append([q,p,finding_pair_q1_q2(f,n)])
        p=P.next(p)              
print(irreducible_finding)                           
        

[[13, 347, [2, 8887]], [13, 349, [2, 39569]], [13, 353, [2, 13597]], [13, 359, [2, 28603]], [13, 367, [2, 4157]], [13, 373, [2, 25073]], [13, 379, [2, 1481]], [13, 383, [2, 6577]], [17, 269, [2, 5303]], [17, 271, [3, 5113]], [17, 277, [2, 28297]], [17, 281, [3, 8447]], [17, 283, [2, 18211]], [17, 293, [2, 25391]], [19, 239, [2, 11071]], [19, 241, [2, 16091]], [19, 251, [2, 66533]], [19, 257, [2, 51383]], [19, 263, [2, 2671]], [23, 197, [2, 7549]], [23, 199, [3, 1697]], [23, 211, [2, 34781]], [29, 157, [2, 797]], [29, 163, [2, 5521]], [29, 167, [2, 4283]], [31, 149, [2, 56681]], [31, 151, [3, 23473]], [31, 157, [2, 18257]], [37, 127, [2, 9533]], [37, 131, [2, 829]], [41, 113, [3, 39901]], [43, 107, [2, 8513]], [43, 109, [2, 26119]], [43, 113, [2, 17923]], [47, 97, [3, 31063]], [47, 101, [2, 1889]], [47, 103, [3, 50587]], [53, 89, [2, 1741]], [59, 79, [2, 40559]], [59, 83, [2, 23321]], [61, 79, [2, 41911]], [67, 71, [2, 52237]], [67, 73, [2, 6121]]]
CPU times: user 2d 6h 33min 3s, sys: 9

In [19]:
%%time
irreducible_finding =[]
n=10**5
lower_range = 5000
upper_range = 5500
for q in q_list:
    p=max(P.next(q), P.next(lower_range//q))    
    while p * q < upper_range: 
        f=fekete(q,p)
        irreducible_finding.append([q,p,finding_pair_q1_q2(f,n)])
        p=P.next(p)              
print(irreducible_finding)                           
        

[[13, 389, [2, 23753]], [13, 397, [2, 12421]], [13, 401, [2, 33619]], [13, 409, [2, 25447]], [13, 419, [2, 3989]], [13, 421, [2, 10433]], [17, 307, [2, 4691]], [17, 311, [3, 10303]], [17, 313, [3, 35831]], [17, 317, [2, 229]], [19, 269, [2, 1669]], [19, 271, [2, 21557]], [19, 277, [2, 8089]], [19, 281, [2, 3677]], [19, 283, [2, 11027]], [23, 223, [3, 2549]], [23, 227, [2, 19183]], [23, 229, [2, 5407]], [23, 233, [3, 7577]], [23, 239, [3, 31259]], [29, 173, [2, 1087]], [29, 179, [2, 5839]], [29, 181, [2, 5557]], [31, 163, [2, 32713]], [31, 167, [3, 6691]], [31, 173, [2, 19387]], [37, 137, [2, 7019]], [37, 139, [2, 14071]], [41, 127, [3, 35747]], [41, 131, [2, 15187]], [43, 127, [2, 80599]], [47, 107, [2, 6719]], [47, 109, [2, 43691]], [47, 113, [3, 61583]], [53, 97, [2, 10009]], [53, 101, [2, 5309]], [53, 103, [2, 20411]], [59, 89, [2, 859]], [61, 83, [2, 7757]], [61, 89, [2, 1213]], [67, 79, [2, 55807]], [71, 73, [3, 10987]]]
CPU times: user 2d 4h 37min 7s, sys: 8min 17s, total: 2d 4h 

In [20]:
%%time
irreducible_finding =[]
n=10**5
lower_range = 5500
upper_range = 6000
for q in q_list:
    p=max(P.next(q), P.next(lower_range//q))    
    while p * q < upper_range: 
        f=fekete(q,p)
        irreducible_finding.append([q,p,finding_pair_q1_q2(f,n)])
        p=P.next(p)              
print(irreducible_finding)                           
        

[[13, 431, [2, 30517]], [13, 433, [2, 2617]], [13, 439, [2, 4229]], [13, 443, [2, 65867]], [13, 449, [2, 4273]], [13, 457, [2, 4337]], [13, 461, [2, 27281]], [17, 331, [2, 22189]], [17, 337, [3, 22013]], [17, 347, [2, 26759]], [17, 349, [2, 45053]], [19, 293, [2, 19501]], [19, 307, [2, 42013]], [19, 311, [2, 39443]], [19, 313, [2, 23567]], [23, 241, [3, 13469]], [23, 251, [2, 8941]], [23, 257, [3, 37879]], [29, 191, [2, 3947]], [29, 193, [2, 7829]], [29, 197, [2, 36583]], [29, 199, [2, 17497]], [31, 179, [2, 9173]], [31, 181, [2, 21283]], [31, 191, [3, 1021]], [31, 193, [3, 1151]], [37, 149, [2, 8627]], [37, 151, [2, 2593]], [37, 157, [2, 6101]], [41, 137, [3, 14011]], [41, 139, [2, 8863]], [43, 131, [2, 6473]], [43, 137, [2, 373]], [43, 139, [2, 5099]], [47, 127, [3, 1823]], [53, 107, [2, 13721]], [53, 109, [2, 4391]], [53, 113, [2, 9941]], [59, 97, [2, 18757]], [59, 101, [2, 41897]], [61, 97, [2, 25939]], [67, 83, [2, 18211]], [67, 89, [2, -1]], [71, 79, [3, 16603]], [71, 83, [2, 556